In [14]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from skimage import data, color, feature
import skimage.data
from sklearn.datasets import fetch_lfw_people

In [15]:
# get images with faces
faces = fetch_lfw_people()
positive_patches = faces.images # obrazki z twarzami
positive_patches.shape

(13233, 62, 47)

In [16]:
# get images without faces
from skimage import data, transform

imgs_to_use = ['camera', 'text', 'coins', 'moon',
               'page', 'clock', 'immunohistochemistry',
               'chelsea', 'coffee', 'hubble_deep_field']

# skimage.data is set of images
images = [color.rgb2gray(getattr(skimage.data, name)()) # Compute luminance of an RGB image for each image
          for name in imgs_to_use]


C:\Users\lewiis\AppData\Local\Temp/ipykernel_8620/577394038.py:9: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  images = [color.rgb2gray(getattr(skimage.data, name)()) # Compute luminance of an RGB image for each image


In [17]:
from sklearn.feature_extraction.image import PatchExtractor

# rescale negative images
def extract_patches(img, N, scale=1.0,
                    patch_size=positive_patches[0].shape):
    extracted_patch_size = tuple((scale * np.array(patch_size)).astype(int))
    extractor = PatchExtractor(patch_size=extracted_patch_size,
                               max_patches=N, random_state=0)
    patches = extractor.transform(img[np.newaxis])
    if scale != 1:
        patches = np.array([transform.resize(patch, patch_size)
                            for patch in patches])
    return patches

negative_patches = np.vstack([extract_patches(img, 1000, scale)
                              for img in images for scale in [0.5, 1.0, 2.0]])
negative_patches.shape

(30000, 62, 47)

In [18]:
from itertools import chain

X_train = np.array([feature.hog(im)
                    for im in chain(positive_patches, negative_patches)]) # extract histograms of oriented gradients

y_train = np.zeros(X_train.shape[0])
y_train[:positive_patches.shape[0]] = 1 # imgaes with faces are marked as 1

In [19]:
# classification model
# network architecture
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3,
                      padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(-1, 64*7*7)
        out = self.layer3(out)
        return out


In [ ]:
# training loop

def train_loop(epochs, optimizer, model, loss_fn, train_loader, val_loader):

    for epoch in range(1, epochs+1):
        loss_train = 0.0
        correct = 0
        total = 0

        for imgs, labels in train_loader:

            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted==labels).sum())

            train_correct = correct
            train_total = total

        epoch_list.append(epoch)
        train_loss_list.append(loss_train)

        # test on validation data
        # get loss of validation data
        with torch.no_grad():
            loss_val = 0.0
            correct = 0
            total = 0
            for imgs, labels in val_loader:
                # move tensors to gpu if available
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)

                outputs = model(imgs)

                loss_v = loss_fn(outputs, labels)

                loss_val += loss_v.item()

                _, predicted = torch.max(outputs, dim=1) # Gives us the index of the highest value
            total += labels.shape[0]  # Counts the number of examples, so total is increased by the batch size
            correct += int((predicted == labels).sum())
            val_correct = correct
            val_total = total


        val_loss_list.append(loss_val / len(val_loader))

        train_acc_list.append(train_correct/train_total)
        val_acc_list.append(val_correct/val_total)

        # set when to print info about training progress
        if epoch == 1 or epoch % 1 == 0:
            print('Epoch {}, Training loss {:.3f}, Validation loss {:.3f}, Train Acc {:.3f}, Val Acc {:.3f}'.format(epoch, loss_train / len(train_loader), loss_val / len(val_loader), train_correct/train_total, val_correct/val_total),
                  )

def test_loop(model, test_loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in test_loader:

            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1) # Gives us the index of the highest value
            total += labels.shape[0]  # Counts the number of examples, so total is increased by the batch size
            correct += int((predicted == labels).sum())


    print("Accuracy test: {:.3f} %".format(100 *  (correct / total)))

In [19]:
# take a test image
test_image = skimage.data.astronaut()
test_image = skimage.color.rgb2gray(test_image)
test_image = skimage.transform.rescale(test_image, 0.5)
test_image = test_image[:160, 40:180]
plt.imshow(test_image, cmap='gray')
plt.axis('off');

In [ ]:
#
def sliding_window(img, patch_size=positive_patches[0].shape, istep=2, jstep=2, scale=1.0):
    Ni, Nj = (int(scale*s) for s in patch_size)
    for i in range(0, img.shape[0] - Ni, istep):
        for j in range(0, img.shape[1] - Ni, jstep):
            patch = img[i:i + Ni, j:j + Nj]
            if scale != 1:
                patch = transform.resize(patch, patch_size)
            yield (i, j), patch

indices, patches = zip(*sliding_window(test_image))
patches_hog = np.array([feature.hog(patch) for patch in patches])
patches_hog.shape

In [ ]:
labels = model.predict(patches_hog)
labels.sum()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(test_image, cmap='gray')
ax.axis('off')
Ni, Nj = positive_patches[1].shape
indices = np.array(indices)

i_ind = []
j_ind = []
for i, j in indices[labels==1]:
    i_ind.append(i)
    j_ind.append(j)

i_ind = np.array(i_ind)
j_ind = np.array(j_ind)

for i, j in indices[labels==1]:
    ax.add_patch(plt.Rectangle((j, i), Nj, Ni, edgecolor='red', alpha=0.3, lw=2, facecolor='none'))
# ax.add_patch(plt.Rectangle((i_ind.mean(), j_ind.mean()), Nj, Ni, edgecolor='red', alpha=0.5, lw=3, facecolor='none'))

# for i, j in indices[labels == 1]:
#     ax.add_patch(plt.Rectangle((j, i), Nj, Ni, edgecolor='red',
#                                alpha=0.3, lw=2,
#                                facecolor='none'))